In [3]:
""" ### I DO NOT HAVE THIS WORKING ### """

from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer

def init_random_num(qc, reg):
    """Randomly initializes an n-qubit quantum register and prints the register label and the binary number."""
    n = len(reg)  # number of qubits in the register
    binary_representation = ''
    
    for qubit in range(n-1): # choose to leave MSB as 0
        # Randomly apply X gate to flip the qubit with 50% probability
        if np.random.rand() > 0.5:
            qc.x(reg[qubit])  # Apply X gate to the quantum circuit
            binary_representation = '1' + binary_representation
        else:
            binary_representation = '0' + binary_representation
            
    binary_representation = '0' + binary_representation
    decimal_value = int(binary_representation, 2)
    
    # Print the label and the binary number created
    print(f"{reg.name}: {binary_representation} (decimal: {decimal_value})") # choose to leave MSB as 0
    
    return qc

def get_decimal_value(counts):
    max_bitstring = max(counts, key=counts.get)
    return int(max_bitstring, 2)

In [40]:
def sqrt(qc, R, F, z, ancillaries):
    """ Info
        This function performs a quantum square root based off the a paper by 
        Edgard Munoz-Coreas, which implements a quantum version of the 
        classical non-restoring sqrt method. We use 3 registers R, F and z.
        
        The function is split into 3 parts: P1, P2, P3

        Parameters:
        - R initialy stores input 'a'
        - F is an empty n bit long register 
        - the input number must have an even length register and be greater than 4
        - qc: QuantumCircuit on which the addition is performed
        - a: QuantumRegister for the number being square rooted

        Returns:
        - QuantumCircuit: The updated quantum circuit where F contains sqrt(a) and 
          R contains the remainder of sqrt(a).

    """
    def ctrl_ADDSUB(qc, a, b, ctrl_value, c):
        """
        Takes in a quantum circuit and two quantum registers (a and b),
        performs an addition or subtraction, and returns the updated circuit.
        It dynamically creates its own carry qubits and removes them 
        before returning the updated circuit. Removals correspond to 
        measurement after the un-compute returns it to 0.
        
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - a: QuantumRegister for the first number
        - b: QuantumRegister for the second number
        - ctrl: Control bit conditioning whether a ADD or SUB is performed
        - c: QuantumRegister for carry qubits
        
        Returns:
        - QuantumCircuit: The updated quantum circuit with the addition operation applied.
        """
    
        n = len(a)  # number of qubits per register
    
        #a flip for subtraction
        for i in a:
            qc.cx(ctrl_value, i)
        
        qc.cx(ctrl_value,a[0])
        qc.cx(ctrl_value,b[0])
        qc.ccx(a[0], b[0], c[0]) 
        qc.cx(ctrl_value,c[0])
    
        #apply repeated segments
        for i in range(1,n-1):
            qc.cx(c[i-1],a[i])
            qc.cx(c[i-1],b[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i-1],c[i])
            
        #end segment (we have no carry-out in this implementation)
        qc.cx(c[n-2],b[n-1]) # n-1 as e.g. n = 4 final position is index=3
    
        #un compute ancillas
        for i in range(n-2,0,-1): # i goes from n-2 to 1
            qc.cx(c[i-1],c[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i-1],a[i])
    
        #uncompute c0 and complete the operation
        qc.cx(ctrl_value,c[0])
        qc.ccx(a[0], b[0], c[0])
        qc.cx(ctrl_value,a[0])
    
        for i in range(0,n):
            qc.cx(a[i],b[i])
        
        #need for b-a as a remains flipped (for a-b) the result is in b so no need to undo.
        for i in a:
            qc.cx(ctrl_value, i)
    
        
        return qc

    def ctrl_add(qc, a, b, ctrl_value, c):
        """
        Takes in a quantum circuit and two quantum registers (a and b),
        performs a controlled addition, and returns the updated circuit.
        
        Parameters:
        - qc: QuantumCircuit on which the addition is performed
        - a: QuantumRegister for the first number
        - b: QuantumRegister for the second number
        - ctrl: Control qubit that determines if the ADD is performed
        - c: QuantumRegister for carry qubits
        
        Returns:
        - QuantumCircuit: The updated quantum circuit with the addition operation applied.
        """
    
        n = len(a)  # number of qubits per register
    
    
        ''' 
        INPUT CARRY/ OUTPUT carry
            This implementation is setup so the first seg is a repeated seg, so if we wanted 
            this adder function to handle input carry:
            1. add Cin as a parameter, 1. use CNOT to initialise c[0] (which otherwise remains 0)
    
            ### add c_in as a parameter and c_out ###
            #bit hard to do those because the uncomputes go back to the last called function for c_in and this call for c_out
    
            # Apply Gidney adder gates
        for i in range(1,n):
            qc.cx(c[i], a[i])
            qc.cx(c[i], b[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i], c[i])
            qc.ccx(ctrl[0], a[i], b[i])
            qc.cx(c[i], a[i])
            qc.cx(c[i], b[i])
        '''
            
       # First seg TempAND
        qc.ccx(a[0], b[0], c[0]) 
    
        # ctrled Gidney adder
        for i in range(1,n-1):
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
            qc.ccx(a[i], b[i], c[i])
            qc.cx(c[i-1], c[i])
    
        # Last Segment
        qc.cx(a[n-1],c[n-2])
        qc.ccx(ctrl_value, c[n-2], b[n-1])
    
        #un compute + last bits
        for i in range(n-2,0,-1): # i goes from n-2 to 1
            #uncompute
            qc.cx(c[i-1], c[i])
            qc.ccx(a[i], b[i], c[i])
    
            qc.ccx(ctrl_value, a[i], b[i])
            qc.cx(c[i-1], a[i])
            qc.cx(c[i-1], b[i])
    
        # End of first
        qc.ccx(a[0], b[0], c[0])
        qc.ccx(ctrl_value, a[0], b[0])
    
        
        return qc


    # number of qubits per register
    n=len(R)

    # Initialise F to 1
    for i in range(0, len(F)):
        qc.x(F)

    """ Part 1 """
    qc.x(R[n-2]) #step 1
    qc.cx(R[n-2], R[n-1]) # step 2
    qc.cx(R[n-1], F[1]) # step 3
    qc.cx(R[n-1], z[0]) # step 4
    qc.cx(R[n-1], F[2]) # step 5
    qc = ctrl_ADDSUB(qc, F[0:3+1], R[n-4:n], z[0], ancillaries) # step 6

    """ Part 2 """ # repeated n/2 - 2 times
    for i in range(2, int(n/2)):

        #step 1
        qc.x(z[0])
        qc.cx(z[0], F[1])
        qc.x(z[0])

        #step 2 
        qc.cx(F[2], z[0])

        #step 3
        qc.cx(R[n-1], F[1])
        
        #step 4
        qc.x(R[n-1])
        qc.cx(R[n-1], z[0])
        qc.x(R[n-1])

        #step 5
        qc.x(R[n-1])
        qc.cx(R[n-1], F[i+1])
        qc.x(R[n-1])

        #step 6 
        
        for j in range(3,i+1+1):
            #SWAP
            qc.cx(F[j],F[j-1])
            qc.cx(F[j-1],F[j])
            qc.cx(F[j],F[j-1])
        """
        if i+1 == 3:
            pass
        else:
            for j in range(3,i+1+1):
                #SWAP
                qc.cx(F[j],F[j-1])
                qc.cx(F[j-1],F[j])
                qc.cx(F[j],F[j-1])
        """
        #step 7 
        qc = ctrl_ADDSUB(qc, F[0:2*i+1+1], R[n-2*i-2:n], z[0], ancillaries)

    """ Part 3 """ 
    #step 1
    qc.x(z[0])
    qc.cx(z[0], F[1])
    qc.x(z[0])

    #step 2
    qc.cx(F[2], z[0])

    #step 3
    qc.x(R[n-1])
    qc.cx(R[n-1], z[0])
    qc.x(R[n-1])

    #step 4
    qc.x(R[n-1])
    qc.cx(R[n-1], F[int(n/2)+1])
    qc.x(R[n-1])

    #step 5 
    qc.x(z[0])

    #step 6 
    qc = ctrl_add(qc, F, R, z[0], ancillaries)

    #step 7
    qc.x(z[0])

    #step 8 
    if n==4:
        pass
    else:
        for j in range(3,int(n/2)+1+1):
            #SWAP
            qc.cx(F[j],F[j-1])
            qc.cx(F[j-1],F[j])
            qc.cx(F[j],F[j-1])
    #step 9
    qc.cx(F[2], z[0])
        
    return qc

In [41]:
list= []
for i in range(0,1):
    list.append(i)
k = len(list)
print(list[k-4:k])

[0]


In [48]:
""" This applies square root to a number """

# Number of bits
n = 6

ancillaries = QuantumRegister(n, 'ancillaries')
F = QuantumRegister(n, 'F') # this contains n for F, n for R and 1 for z
z = QuantumRegister(1, 'z')
a = QuantumRegister(n, 'a')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(2*n, 'result')

qc = QuantumCircuit(a, F, z, result, ancillaries)

"""initialize the quantum numbers"""
qc = init_random_num(qc, a)

# a = 36 (100100)
#qc.x(a[5])
#qc.x(a[2])


#apply the ctrl add/sub operation
qc = sqrt(qc, a, F, z, ancillaries)

# Measure the sum qubits and carry qubit
#qc.measure_all()

# Measure the square root and remainder
#qc.measure(F[2:int(n/2)+2], result[:int(n/2)])   # Measure sqrt(a) in 'F'
qc.measure(F, result[:n]) # measure the F register into the first n classical bits
qc.measure(a, result[n:2*n+n])     # Measure remainder 'R' in 'a'


# For execution
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)

def binary_fraction_to_decimal(binary_str):
    """Convert a binary fraction string to its decimal equivalent."""
    decimal_value = 0
    for i, digit in enumerate(binary_str):
        if digit == '1':
            decimal_value += 2 ** -(i + 1)
    return decimal_value

print("Measurement results:")
for bitstring, count in counts.items():
    sqrt_result = bitstring[-int(n/2)-2:-2]        # First half of the result is the sqrt(a)
    remainder_result = bitstring[-3*n:-n]   # Second half is the remainder R
    
    sqrt_decimal_value = int(sqrt_result, 2)
    remainder_decimal_value = binary_fraction_to_decimal(remainder_result)
        
    print(f"sqrt(a) = {sqrt_result} (decimal: {sqrt_decimal_value}), R = 0.{remainder_result} (decimal: {remainder_decimal_value:.6f})")
    # Draw the circuit (we can't draw the circuit if we remove the qubits)
#qc_drawn = circuit_drawer(qc, output='text')
#print(qc_drawn)

a: 000101 (decimal: 5)
{'001111011110': 1024}
Measurement results:
sqrt(a) = 111 (decimal: 7), R = 0.001111 (decimal: 0.234375)


In [13]:
sqrt_reg = QuantumRegister(2*n+1, 'sqrt_reg')

qc = QuantumCircuit( sqrt_reg, )

f = sqrt_reg[0:n]
z = sqrt_reg[n:n+1]

qc.x(z[0])




In [86]:
# Number of bits
n = 6 

a = QuantumRegister(n, 'a')

# Define a classical register with 2n bits (n for a and n for b)
result = ClassicalRegister(n, 'result')

qc = QuantumCircuit(a)

# a = 36 (100100)
qc.x(a[5])
qc.x(a[2])

qc.measure_all()

simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
sim_result = simulator.run(compiled_circuit).result()
counts = sim_result.get_counts()

print(counts)


{'100100': 1024}
